In [ ]:
import param
import numpy as np
import holoviews as hv
from holoviews import opts

hv.extension('bokeh', 'matplotlib')

# Linked Streams

There are several pre-defined linked stream classes provided by HoloViews which allow you to create interactions based on user actions in the browser. Many of these events occur natively in the browser and are available in JavaScript code. Bokeh, however, is more than a simple interactive plotting tool. There is a server component that can maintain 2-way communication with the generated plots. It captures events from the browser and transmits them back to the listening server. HoloViews provides these linked Stream classes and provides a way for you to define custom actions to take in response to user actions. A selection of them are shown here:

* Plot Ranges
  - `RangeX`, `RangeY`, and `RangeXY`
* Pointer Location
  - `PointerX`, `PointerY`, `PointerXY`
* Mouse Events
  - `Tap`, `DoubleTap`, `MouseEnter`, `MouseLeave`
* Selections
  - `Selection1D`, `BoundsX`, `BoundsXY`, `BoundsY`
* PlotSize
  - `PlotSize`
* Drawing
  - `BoxEdit`, `PointDraw`, `FreehandDraw`, `PolyDraw`, `PolyEdit`

In [ ]:
# Create a stream to watch the mouse pointer
pointer = hv.streams.PointerXY()
# create a DynamicMap with a function to plot a single point and listen to our Stream
pointer_dmap = hv.DynamicMap(lambda x, y: hv.Points([(x, y)]), streams=[pointer])
pointer_dmap.opts(size=10)

We can even examine the current contents of the `pointer` stream.

In [ ]:
pointer.contents

# Explicit Linking

In the example above, the source of the stream was automatically linked to the DynamicMap. We can also explicitly set the source. In this example we create two plots and set the source to one plot while using the coordinates to update the other plot.

In [ ]:
# Create a 2D image
xvals = np.linspace(0,4,202)
ys,xs = np.meshgrid(xvals, -xvals[::-1])
img = hv.Image(np.sin(((ys)**3)*xs))

# create a pointer and set its source
pointer = hv.streams.PointerXY(x=0,y=0, source=img)

# create plots
pointer_dmap = hv.DynamicMap(lambda x, y: hv.Points([(x, y)]), streams=[pointer])
pointer_dmap = pointer_dmap.redim.range(x=(-0.5,0.5), y=(-0.5,0.5))

# compose plots
img + pointer_dmap.opts(size=10)

**This also works across cells in a jupyter notebook.**

In [ ]:
# Create a 2D image
xvals = np.linspace(0,4,202)
ys,xs = np.meshgrid(xvals, -xvals[::-1])
img = hv.Image(np.sin(((ys)**3)*xs))

# create a pointer and set its source
pointer = hv.streams.PointerXY(x=0,y=0, source=img)

# create plots
pointer_dmap = hv.DynamicMap(lambda x, y: hv.Points([(x, y)]), streams=[pointer])
pointer_dmap = pointer_dmap.redim.range(x=(-0.5,0.5), y=(-0.5,0.5))

img

In [ ]:
pointer_dmap.opts(size=10)

# Transient Streams

Transient streams only have values when the event occurs and fall back to the default otherwise (usually `None`). 

In [ ]:
# create two streams to listen to Tap and DoubleTap events
tap = hv.streams.SingleTap(transient=True)
double_tap = hv.streams.DoubleTap(rename={'x': 'x2', 'y': 'y2'}, transient=True)

# track each event and return plot with updated points
taps = []
def record_taps(x, y, x2, y2):
    if None not in [x,y]:
        taps.append((x, y, 1))
    elif None not in [x2, y2]:
        taps.append((x2, y2, 2))
    return hv.Points(taps, vdims='Taps')

# Provide DynamicMap with streams
taps_dmap = hv.DynamicMap(record_taps, streams=[tap, double_tap])

# Plot
taps_dmap.opts(color='Taps', cmap={1: 'red', 2: 'gray'}, size=10, tools=['hover'])

In [ ]:
taps